In [331]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss
import xgboost as xgb
import functools as ft
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\PDudukin\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['test', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [332]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

cols = train.columns
cols = cols.drop(['listing_id','interest_level'])
train_X = train[cols]
train_y = train['interest_level']
test_X = test[cols]

train_data = pd.concat([train_X, test_X], axis=0)

### Features Engineering

In [333]:
cols_to_feat = ['features', 'description', 'photos']

for col in cols_to_feat:
    train_data['n_'+col] = train_data[col].apply(len)
    
train_data['log_price'] = train_data['price'].apply(np.log)

#### Work woth Created Data

In [334]:
train_data.created = pd.to_datetime(train_data.created)
train_data['date'] = [d.date() for d in train_data.created]
train_data['time'] = [d.time() for d in train_data.created]

In [335]:
train_data['day'] = [d.day for d in train_data.date]
train_data['month'] = [d.month for d in train_data.date]

train_data['hour'] = [d.hour for d in train_data.time]
train_data['hms'] = [d.hour+d.minute/60.+d.second/3600. for d in train_data.time]

In [336]:
def hex_to_int(x):
    return eval("0x" + x)

In [337]:
a = "5ba989232d0489da1b5f2c45f6688adc"
b = "7533621a882f71e25173b27e3139d83d"
c = "d9039c43983f6e564b1482b273bd7b01"

a1 = hex_to_int(a)
b1 = hex_to_int(b)
c1 = hex_to_int(c)

#sum(a1), b1, c1
sum(int(digit) for digit in str(a1)), sum(int(digit) for digit in str(b1)), sum(int(digit) for digit in str(c1))

(156, 188, 188)

In [338]:
categorical = ["display_address", "manager_id", "building_id", "street_address"]
features_to_use = []

for f in categorical:
        if train_X[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_data[f].values))
            
            train_X[f] = lbl.transform(list(train_X[f].values))
            test_X[f] = lbl.transform(list(test_X[f].values))
            
            train_data[f] = lbl.transform(list(train_data[f].values))
            
            features_to_use.append(f)

C:\Users\PDudukin\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\PDudukin\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [339]:
features_to_use

['display_address', 'manager_id', 'building_id', 'street_address']

In [340]:
train_data.columns

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'latitude', 'longitude', 'manager_id',
       'photos', 'price', 'street_address', 'n_features', 'n_description',
       'n_photos', 'log_price', 'date', 'time', 'day', 'month', 'hour', 'hms'],
      dtype='object')

In [341]:
train_data.head(3)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,longitude,manager_id,...,n_features,n_description,n_photos,log_price,date,time,day,month,hour,hms
10,1.5,3,3797,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,12282,[],40.7145,-73.9425,1568,...,0,588,5,8.006368,2016-06-24,07:54:24,24,6,7,7.906667
10000,1.0,2,8986,2016-06-12 12:19:27,,9080,"[Doorman, Elevator, Fitness Center, Cats Allow...",40.7947,-73.9667,1988,...,5,8,11,8.606119,2016-06-12,12:19:27,12,6,12,12.324167
100004,1.0,1,8889,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",13719,"[Laundry In Building, Dishwasher, Hardwood Flo...",40.7388,-74.0018,3733,...,4,691,8,7.955074,2016-04-17,03:26:41,17,4,3,3.444722


In [342]:
cols_to_delete = ['created','photos', 'price', 'features', 'description', 'date', 'time']
train_data = train_data.drop(cols_to_delete, axis = 1)

In [343]:
train_data.head(2)

,bathrooms,bedrooms,building_id,display_address,latitude,longitude,manager_id,street_address,n_features,n_description,n_photos,log_price,day,month,hour,hms
10,1.5,3,3797,12282,40.7145,-73.9425,1568,23484,0,588,5,8.006368,24,6,7,7.906667
10000,1.0,2,8986,9080,40.7947,-73.9667,1988,23680,5,8,11,8.606119,12,6,12,12.324167


In [344]:
print(train_X.shape[0])
print(test_X.shape[0])
print(train_data.shape[0])

49352
74659
124011


In [345]:
train_X = train_data[:train_X.shape[0]]
train_y = train['interest_level'][:train_X.shape[0]]

test_X = train_data[train_X.shape[0]:]

In [346]:
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)

(49352, 16)
(49352,)
(74659, 16)


In [347]:
#xgb.fit(train_X,train_y)

In [348]:
rft = RandomForestClassifier(n_jobs=-1)
params = {
    'n_estimators' : [50,100,200,400],
    'max_features' : ['auto'],
    'max_depth' : [3,5,10,20,50],
    'min_samples_leaf' : [0.003],
    'min_samples_split' : [0.001],
    'criterion' : ['entropy','gini'],
    'class_weight' : ['balanced',None],
    'bootstrap' : [True],
    'oob_score' : [False],
    'random_state' : [0,123,12345]
}
def framework(clf,params,n_iter):
    # calculate # of iterations for Search
    parsize = ft.reduce(lambda a,b: a*b,[len(params[x]) for x in params]) # total # of combinations
    psize = n_iter if parsize > n_iter else parsize # limit # by n_iter
    print ('Parameters combination :',str(psize)+"/"+str(parsize))   
    
    rgs = RandomizedSearchCV(
    estimator = clf,
    param_distributions = params,
    n_iter = psize,
    scoring = 'neg_log_loss',
    n_jobs = -1,
    cv = 5,
    refit=True,
    verbose=1)

    rgs.fit(train_X,train_y)
    
    n_top = 3
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(rgs.cv_results_['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i), "(best model)" if rgs.best_index_ == candidate else "")
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  rgs.cv_results_['mean_test_score'][candidate],
                  rgs.cv_results_['std_test_score'][candidate]))
            print("Parameters: {0}".format(rgs.cv_results_['params'][candidate]))
    
    return rgs.best_estimator_

In [349]:
xgb = xgb.XGBClassifier(nthread=-1)
#print(xgb.get_params().keys())
xgb_params = {"max_depth": [3,5,7],
              "learning_rate": [0.01,0.05,0.1,0.15],
              "n_estimators": [50,75,100,150,200],
              "min_child_weight": [0.01,0.005],
              "gamma": [0.1,0.5,0.8,0.9,1.0],
              "subsample":[0.75,0.9,1.0],
              #"eval_metric":['logloss'],
              "objective":["multi:softprob"],
              #"seed":[0],
              "nthread":[-1]
              #,"eval_metric": ['roc_auc']
             }

est = framework(xgb,xgb_params,50)


#Model with rank: 1 (best model)
#Mean validation score: -0.597 (std: 0.005)
#Parameters: {'subsample': 0.9, 'objective': 'multi:softprob', 'nthread': -1, 'n_estimators': 100, 'min_child_weight': 0.005, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0.8}

Parameters combination : 50/1800
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 33.1min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 40.0min finished


Model with rank: 1 (best model)
Mean validation score: -0.572 (std: 0.005)
Parameters: {'subsample': 0.9, 'objective': 'multi:softprob', 'nthread': -1, 'n_estimators': 150, 'min_child_weight': 0.01, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0.8}
Model with rank: 2 
Mean validation score: -0.573 (std: 0.005)
Parameters: {'subsample': 0.75, 'objective': 'multi:softprob', 'nthread': -1, 'n_estimators': 150, 'min_child_weight': 0.005, 'max_depth': 7, 'learning_rate': 0.15, 'gamma': 0.8}
Model with rank: 3 
Mean validation score: -0.573 (std: 0.004)
Parameters: {'subsample': 1.0, 'objective': 'multi:softprob', 'nthread': -1, 'n_estimators': 200, 'min_child_weight': 0.005, 'max_depth': 5, 'learning_rate': 0.15, 'gamma': 0.1}


FileNotFoundError: [Errno 2] No such file or directory: './scores/scored_hypopt.csv'

In [372]:
result3 = pd.DataFrame(est.predict_proba(test_X),columns=est.classes_)
result3['listing_id'] = test['listing_id'].values


In [373]:
result3

,high,low,medium,listing_id
0,0.074025,0.420610,0.505365,7142618
1,0.127660,0.795146,0.077195,7210040
2,0.034733,0.834971,0.130296,7103890
3,0.066747,0.473859,0.459394,7143442
4,0.022298,0.803688,0.174013,6860601
5,0.001916,0.947779,0.050305,6840081
6,0.049108,0.686517,0.264376,6922337
7,0.126246,0.443819,0.429935,6913616
8,0.099785,0.549653,0.350562,6937820
9,0.027569,0.706633,0.265798,6893933


In [374]:
result3[['listing_id','high','medium','low']].to_csv('scores_2.csv',index=False)